<p>Write a program using Lists and loops. Make sure to test for any possible error and give user-friendly messages; make your testing plan clear and visible through comments for both of the coding below.
<br /><br />
1. Standard telephone keypads contain the digits zero through nine. The numbers two through nine each have three letters associated with them, as shown in the following table (look on your phone for the rest of the mapping of 5 through 9):
<br /><br />
Digit Letters<br />
2      АBC<br />
3      DEF<br />
4      GHI<br /><br />
Many people find it difficult to memorize phone numbers, so they use the correspondence between digits and letters to develop seven-letter words (or phrases) that correspond to their phone numbers. For example, a person whose telephone number is 686-2377 might use the correspondence indicated in the preceding table to develop the seven-letter word "NUMBERS.” Every seven-letter word or phrase corresponds to exactly one seven-digit telephone number. A budding data science entrepreneur might like to reserve phone number 244-3282 (“BIGDATA"). Every seven-digit phone number without Os or 1s corresponds to many different seven-letter words, but most of these words represent unrecognizable gibberish. A veterinarian with phone number 738-2273 would be pleased to know that the number corresponds to the letters “PETCARE.” Write a script that, given a seven-digit number, generates every possible seven-letter word combination corresponding to that number. There are 2,187 (37) such combinations. Avoid phone numbers with the digits 0 and 1 (to which no letters correspond). See if your phone number corresponds to meaningful words.</p>

In [1]:
import re
import nltk
from nltk.corpus import wordnet
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/jking/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# Setting up a constant reusable map of digit : n-alphas
NUMBER_MAP = {0: [], 1: [], 2: ['A', 'B', 'C'], 3: ['D', 'E', 'F'], 
              4: ['G', 'H', 'I'], 5: ['J', 'K', 'L'], 6: ['M', 'N', 'O'], 
              7: ['P', 'Q', 'R', 'S'], 8: ['T', 'U', 'V'],
              9: ['W', 'X', 'Y', 'Z']}

In [3]:
# Numbers for testing
number = '244-7464'  # Big ring
# number = '686-2377'  # Numbers
# number = '244-3282'  # Big data
# number = '738-2273'  # Pet care
# number = '909-8404' # My number is not ideal :(
# number = '2222'  # Testing junk input number

### Setup an output dictionary to target

In [4]:
if '0' in number or '1' in number:
    print('The input number contains 0 and 1s, try another')
    output = {}
elif len(number) != 8:
    print('The number must be characters long in the XXX-XXXX format')
    output = {}
else:
    # Apply regular expresion to ensure we only have 2-9 numerics
    numbers = re.sub('[^2-9]', '', number)
    output = dict()
    counter = 0
    # For each digit n in the number
    for n in list(numbers):
        # Fill a new dict with digit position : possible vanity alphas
        output[counter] = NUMBER_MAP.get(int(n))
        counter += 1

In [5]:
output

{0: ['A', 'B', 'C'],
 1: ['G', 'H', 'I'],
 2: ['G', 'H', 'I'],
 3: ['P', 'Q', 'R', 'S'],
 4: ['G', 'H', 'I'],
 5: ['M', 'N', 'O'],
 6: ['G', 'H', 'I']}

### Brute-force nested loop solution
<p>There are scalability issues here as there is an assumption that there will always be 6 levels. It also looks terrible, but for the sake of assignment I feel like it illustrates the solution clearly.</p>

In [6]:
combos = []
if len(output) > 0:
    # For each alpha associated with each digit...
    for a in output.get(0):
        for a1 in output.get(1):
            for a2 in output.get(2):
                for a3 in output.get(3):
                    for a4 in output.get(4):
                        for a5 in output.get(5):
                            for a6 in output.get(6):
                                # Concat all six levels of alphas and append the lowercase string to combos
                                combos.append((a + a1 + a2 + a3 + a4 + a5 + a6).lower())
print(combos[0:10])
print(combos[-10:])
print(len(combos))

['aggpgmg', 'aggpgmh', 'aggpgmi', 'aggpgng', 'aggpgnh', 'aggpgni', 'aggpgog', 'aggpgoh', 'aggpgoi', 'aggphmg']
['ciishoi', 'ciisimg', 'ciisimh', 'ciisimi', 'ciising', 'ciisinh', 'ciisini', 'ciisiog', 'ciisioh', 'ciisioi']
2916


### A more elegant recursive solution
<p>The above code has been "simplified" by implementing the following recursive function.  It selects the level of recursion dynamically based on the size of the output dictionary, solving the limitations of the former solution.</p>

In [7]:
def get_combos(combo='', output={}, idx=0, combo_list=[]):
    """
    Get all combinations of alphas for a six digit phone number from the output dict, recursively
    :param combo: Combination string, starting with ''
    :param output: Output dict in the form of {digit_slot: [list, of, alphas]}
    :param idx: Index value recursive iteration, starting with 0
    :return: list of combination strings
    """
    # Start the outer iteration of the top k,v pair
    for alpha in output.get(idx):
        if idx < len(output)-1:
            # If idx is less than the total number of indices to iteration, then call self, incrementing
            get_combos(combo=combo+alpha, output=output, idx=idx+1, combo_list=combo_list)
        elif idx == len(output)-1:
            # Once idx is equal to the total recursion level, append the string upon the final iteration
            combo_list.append((combo+alpha).lower())
    return combo_list

In [8]:
combos = []
if len(output) > 0:
    combos = get_combos(combo='', output=output, combo_list=[])
print(combos[0:10])
print(combos[-10:])
print(len(combos))

['aggpgmg', 'aggpgmh', 'aggpgmi', 'aggpgng', 'aggpgnh', 'aggpgni', 'aggpgog', 'aggpgoh', 'aggpgoi', 'aggphmg']
['ciishoi', 'ciisimg', 'ciisimh', 'ciisimi', 'ciising', 'ciisinh', 'ciisini', 'ciisiog', 'ciisioh', 'ciisioi']
2916


### Extra Credit
<p>I was curious what words or multipart word phases I would find so I thought I would take it further to see what words I could identify.  I selected the NLTK <a href="https://www.nltk.org/howto/wordnet.html">WordNet Interface</a> as a way to identify full words and word paris</p>

In [9]:
word_combos = []
# For all combinations in the combos list
for c in combos:
    # If the entire combo is found in wordnet keep it
    if wordnet.synsets(c):
        word_combos.append(c)
    
    # Split the comobs into XXX XXX and check each part
    part_one = c[:3]
    part_two = c[3:]
    if wordnet.synsets(part_one) and wordnet.synsets(part_two):
        word_combos.append(f'{part_one} {part_two}')

word_combos = sorted(word_combos, key=len)
print(word_combos)

['chiping', 'big ping', 'big qing', 'big ring', 'big sing', 'chi ping', 'chi qing', 'chi ring', 'chi sing']


<hr />

<p>2.Just as people would enjoy knowing what word or phrase their phone number corresponds to, they might choose a word or phrase appropriate to their business and determine what phone numbers correspond to it. These are sometimes called vanity phone numbers, and various websites sell such phone numbers. Write a script similar to the above that produces the possible phone number for the given seven letter string.</p>

In [10]:
def get_vanity_number_from_string(vanity_string):
    """
    Get a vanity number for the associated string value
    :param vanity_string: Desired vanity string value
    :return: string in the format of XXX-XXXX
    """
    number = []
    # Check if the input string is all alpha characters
    if not vanity_string.isalpha():
        print(f'vanity_string must be alpha, {vanity_string} is not.')
        number = ""
    # Check if the input string does not contain 6 characters
    elif len(vanity_string) != 7:
        print(f'vanity_string must be 7 characters in length, "{vanity_string}" is {len(vanity_string)}.')
        number = ""
    else:
        # For all uppercase alpha chacters in the input string
        for vanity_alpha in list(vanity_string.upper()):
            # For all digit / alphas
            for digit, alphas in NUMBER_MAP.items():
                if vanity_alpha in alphas:
                    number.append(digit)
        # Flatten the list of digist into a single string
        number = "".join([str(x) for x in number])
        # Format in number format as XXX-XXXX
        number = f'{number[:3]}-{number[3:]}'
    return number

In [11]:
get_vanity_number_from_string(vanity_string='petcase')

'738-2273'

In [12]:
# Test for non-alpha check
number = get_vanity_number_from_string(vanity_string='bigd@t@')
assert len(number)==0, 'Non-alpha test failed'
# Test for non-7 character string
number = get_vanity_number_from_string(vanity_string='aabbccdd')
assert len(number)==0, 'Non-7 chacater test failed'

vanity_string must be alpha, bigd@t@ is not.
vanity_string must be 7 characters in length, "aabbccdd" is 8.
